In [2]:
%matplotlib inline
from scipy import io
import numpy as np
import matplotlib.pyplot as plt

In [17]:
monkey_ecog_data = io.loadmat('ECoG_monkey/ECoG_monkey.mat', squeeze_me = True)

In [20]:
print(monkey_ecog_data['elec_num'])

[52 70 91]
